## Movie Recommender System

In this notebook, I will be building a movie recommender system using Python and Pandas. 
I will be using the Collabrotive-Filtering approach to build the system. More specifically, I will be using the Memory Based Filtering approach as well as the Model Based filtering approach to get my results.

The Memory-Based CF approach will use cosine similairty while the Model Based CF approach will use singular value decomposition. Collaborative Filtering recommends items based on similarity measures between users and/or items. 

The data used for this recommender system is the popular MovieLens dataset, which is one of the most common datasets used when implementing and testing recommender engines. It contains 100k movie ratings from 943 users and a selection of 1682 movies.

First let's import the libraries we will be using and load the data

# Getting Started

In [1]:
import numpy as np
import pandas as pd

#Reading the dataset
column_names = ['user_id', 'item_id', 'rating', 'timestamp']
df = pd.read_csv('u.data', sep='\t', names=column_names)

Now lets take a look at the head of the data

In [2]:
df.head()

,user_id,item_id,rating,timestamp
0,0,50,5,881250949
1,0,172,5,881250949
2,0,133,1,881250949
3,196,242,3,881250949
4,186,302,3,891717742


Lets grab the movie titles since we don't have that data

In [3]:
movie_titles = pd.read_csv("Movie_Id_Titles")
movie_titles.head(10)

,item_id,title
0,1,Toy Story (1995)
1,2,GoldenEye (1995)
2,3,Four Rooms (1995)
3,4,Get Shorty (1995)
4,5,Copycat (1995)
5,6,Shanghai Triad (Yao a yao yao dao waipo qiao) ...
6,7,Twelve Monkeys (1995)
7,8,Babe (1995)
8,9,Dead Man Walking (1995)
9,10,Richard III (1995)


Now let's merge the movie titles with our dataframe using the item_id column

In [4]:
df = pd.merge(df,movie_titles,on='item_id')
df.head(20)

,user_id,item_id,rating,timestamp,title
0,0,50,5,881250949,Star Wars (1977)
1,290,50,5,880473582,Star Wars (1977)
2,79,50,4,891271545,Star Wars (1977)
3,2,50,5,888552084,Star Wars (1977)
4,8,50,5,879362124,Star Wars (1977)
5,274,50,5,878944679,Star Wars (1977)
6,227,50,4,879035347,Star Wars (1977)
7,99,50,5,885679998,Star Wars (1977)
8,305,50,5,886321799,Star Wars (1977)
9,108,50,4,879879739,Star Wars (1977)


Lets take a look at the number of unique movie titles and users

In [5]:
n_users = df.user_id.nunique()
n_items = df.item_id.nunique()

print('Num. of Users: '+ str(n_users))
print('Num of Movies: '+str(n_items))

Num. of Users: 944
Num of Movies: 1682


# Evaluation

To evaluate the recommender system, lets split the data into two sets using the Train test split. 

In [6]:
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(df, test_size=0.25)

# Memory Based Filtering

This approach can be divided into two sections: user-item filtering and item-item filtering.

*User-item filtering* takes a particular user, finds users that are similar based on similarity of ratings, recommend items that those similar users liked. 

In contrast, *item-item filtering* will take an item, find users who liked that item, and find other items that those users or similar users also liked. It takes items and outputs other items as recommendations. 

In both cases, I will create a user-item matrix with the entire dataset. 

The dataset has been split into two: train_data and test_data so I will need to create two matrices. The size of the matrices will be [944,1682]. The training matrix will contain 75% of ratings and the testing matrix will contain 25%.

Lets create the two matrices.

In [7]:
#Matrix 1 for training data
train_data_matrix = np.zeros((n_users, n_items))
for line in train_data.itertuples():
    train_data_matrix[line[1]-1, line[3]-1] = line[3]

#Matrix 2 for testing data
test_data_matrix = np.zeros((n_users, n_items))
for line in test_data.itertuples():
    test_data_matrix[line[1]-1, line[2]-1] = line[3]


A distance metric commonly used in recommender systems is cosine similarity, where the ratings are seen as vectors in n-dimensional space and the similarity is calculated based on the angle between these vectors. Cosine similiarity for users a and m can be calculated using the formula below, where you take dot product of the user vector  𝑢𝑘 and the user vector  𝑢𝑎 and divide it by multiplication of the Euclidean lengths of the vectors.

<img class="aligncenter size-thumbnail img-responsive" src="https://latex.codecogs.com/gif.latex?s_u^{cos}(u_k,u_a)=\frac{u_k&space;\cdot&space;u_a&space;}{&space;\left&space;\|&space;u_k&space;\right&space;\|&space;\left&space;\|&space;u_a&space;\right&space;\|&space;}&space;=\frac{\sum&space;x_{k,m}x_{a,m}}{\sqrt{\sum&space;x_{k,m}^2\sum&space;x_{a,m}^2}}"/>

In the memory-based approach, the cosine similarity between the users to find out the users with similar interests, larger cosine implies that there is a smaller angle between two users, hence they have similar interests.

Lets use the pairwise funciton in sklearn to calculate the cosine similarity.

In [8]:
from sklearn.metrics.pairwise import pairwise_distances
user_similarity = pairwise_distances(train_data_matrix, metric='cosine')
item_similarity = pairwise_distances(train_data_matrix.T, metric='cosine')

Next step is to make predictions. The similary matrices have been created as user_similarity and item_similarity so I can now predict based on the user-based CF formula.

<img class="aligncenter size-thumbnail img-responsive" src="https://latex.codecogs.com/gif.latex?\hat{x}_{k,m}&space;=&space;\bar{x}_{k}&space;&plus;&space;\frac{\sum\limits_{u_a}&space;sim_u(u_k,&space;u_a)&space;(x_{a,m}&space;-&space;\bar{x_{u_a}})}{\sum\limits_{u_a}|sim_u(u_k,&space;u_a)|}"/>

For user based CF, the similarity between the two users in the formula needs to be normalized so that the ratings stay between 1 and 5 and, as a final step, I will need to sum the average ratings for the user that you are trying to predict. 

The idea here is that some users may tend always to give high or low ratings to all movies. The relative difference in the ratings that these users give is more important than the absolute values. To give an example: suppose, user k gives 4 stars to his favourite movies and 3 stars to all other good movies. Suppose now that another user t rates movies that he/she likes with 5 stars, and the movies he/she fell asleep over with 3 stars. These two users could have a very similar taste but treat the rating system differently.

When making a prediction for item-based CF you don't need to correct for users average rating since query user itself is used to do predictions

<img class="aligncenter size-thumbnail img-responsive" src="https://latex.codecogs.com/gif.latex?\hat{x}_{k,m}&space;=&space;\frac{\sum\limits_{i_b}&space;sim_i(i_m,&space;i_b)&space;(x_{k,b})&space;}{\sum\limits_{i_b}|sim_i(i_m,&space;i_b)|}"/>

In [9]:
def predict(ratings, similarity, type='user'):
    if type == 'user':
        #Get the mean of ratings
        mean_user_rating = ratings.mean(axis=1)
        #Use np.newaxis so that mean_user_rating has same format as ratings
        ratings_diff = (ratings - mean_user_rating[:, np.newaxis])
        #Add the mean 
        pred = mean_user_rating[:, np.newaxis] + similarity.dot(ratings_diff) / np.array([np.abs(similarity).sum(axis=1)]).T
    elif type == 'item':
        pred = ratings.dot(similarity) / np.array([np.abs(similarity).sum(axis=1)])
    return pred

In [11]:
user_prediction = predict(train_data_matrix, user_similarity, type='user')
item_prediction = predict(train_data_matrix, item_similarity, type='item')

print(item_prediction)
print(user_prediction)

[[1.29118372e-03 4.33209054e-04 2.30294348e-04 ... 8.92325996e-03
  8.92325996e-03 8.92325996e-03]
 [1.29118372e-03 4.33209054e-04 2.30294348e-04 ... 8.92325996e-03
  8.92325996e-03 8.92325996e-03]
 [1.29118372e-03 4.33209054e-04 2.30294348e-04 ... 8.92325996e-03
  8.92325996e-03 8.92325996e-03]
 ...
 [1.10677853e-03 3.40988661e-04 9.27064997e-05 ... 7.13860797e-03
  7.13860797e-03 7.13860797e-03]
 [1.29118372e-03 4.33209054e-04 2.30294348e-04 ... 8.92325996e-03
  8.92325996e-03 8.92325996e-03]
 [4.68726700e-04 2.63173920e-04 1.60801403e-04 ... 3.56930399e-03
  3.56930399e-03 3.56930399e-03]]
[[ 3.69384319e-01  1.39466372e+00  2.54534559e+00 ...  2.69678633e-03
   2.69678633e-03  2.69678633e-03]
 [ 3.69384319e-01  1.39466372e+00  2.54534559e+00 ...  2.69678633e-03
   2.69678633e-03  2.69678633e-03]
 [ 3.69384319e-01  1.39466372e+00  2.54534559e+00 ...  2.69678633e-03
   2.69678633e-03  2.69678633e-03]
 ...
 [ 7.17860463e-01  1.94081707e+00  2.86880616e+00 ... -8.69693485e-04
  -8.69693

To evaluate the accuracy of the predicted ratings

In [12]:
from sklearn.metrics import mean_squared_error
from math import sqrt
def rmse(prediction, ground_truth):
    prediction = prediction[ground_truth.nonzero()].flatten()
    ground_truth = ground_truth[ground_truth.nonzero()].flatten()
    return sqrt(mean_squared_error(prediction, ground_truth))

In [13]:
print('User-based CF RMSE: ' + str(rmse(user_prediction, test_data_matrix)))
print('Item-based CF RMSE:' + str(rmse(item_prediction, test_data_matrix)))

User-based CF RMSE: 3.6865442211762627
Item-based CF RMSE:3.6891931532904922


# Conclusion

To conclude, the low user based RMSE indicates that the recommender system accurately predicts ratings by leveraging user-user similarities effectively and the low item based RMSE indicates that the recommender system accurately predicts ratings by leveraging item-item similarities effectively.

However, considerations such as sparsity, scale and distribution of ratings can make RMSE to be less informative on its own and should be interpreted alongside other metrics. Model-based CF methods are scalable and can deal with higher sparsity level than memory-based models, but also suffer when new users or items that don't have any ratings enter the system.

# Model-based Collaborative Filtering

The model based CF approach uses statistical methods to learn patterns and make predictions based on those patterns. One of these methods are **matrix factorization(MF)**. MF can deal better with scalabilty and sparsity for recommender systems better than Memory Based. MF decomposes the user-item interaction matrix into lower-dimensional matrices, revealing latent factors for users and items.

Let's calculate the sparsity level of MovieLens dataset:

In [14]:
sparsity = sparsity=round(1.0-len(df)/float(n_users*n_items),3)
print('The sparsity level of MovieLens is ' +  str(sparsity*100) + '%')

The sparsity level of MovieLens is 93.7%


The learned latent preferences of the user can allow the model to learn which data is most important on its own even if that data is not a variable. The important aspect is that the CF model only uses data (user_id, movie_id, rating) to learn the latent features. If there is little data available model-based CF model will predict poorly, since it will be more difficult to learn the latent features.

# SVD

The general equation can be expressed as follows:
<img src="https://latex.codecogs.com/gif.latex?X=USV^T" title="X=USV^T" />


Given `m x n` matrix `X`:
* *`U`* is an *`(m x r)`* orthogonal matrix
* *`S`* is an *`(r x r)`* diagonal matrix with non-negative real numbers on the diagonal
* *V^T* is an *`(r x n)`* orthogonal matrix

Elements on the diagnoal in `S` are known as *singular values of `X`*. 

Matrix *`X`* can be factorized to *`U`*, *`S`* and *`V`*. The *`U`* matrix represents the feature vectors corresponding to the users in the hidden feature space and the *`V`* matrix represents the feature vectors corresponding to the items in the hidden feature space.
<img class="aligncenter size-thumbnail img-responsive" style="max-width:100%; width: 50%; max-width: none" src="http://s33.postimg.org/kwgsb5g1b/BLOG_CCA_5.png"/>

Now I can make a prediction by taking dot product of *`U`*, *`S`* and *`V^T`*.

<img class="aligncenter size-thumbnail img-responsive" style="max-width:100%; width: 50%; max-width: none" src="http://s33.postimg.org/ch9lcm6pb/BLOG_CCA_4.png"/>

In [15]:
import scipy.sparse as sp
from scipy.sparse.linalg import svds

#get SVD components from train matrix. Choose k.
u, s, vt = svds(train_data_matrix, k = 20)
s_diag_matrix=np.diag(s)
X_pred = np.dot(np.dot(u, s_diag_matrix), vt)
print('User-based CF MSE: ' + str(rmse(X_pred, test_data_matrix)))

User-based CF MSE: 3.6870552240169934


# Conclusion

In this notebook, we have seen two collaborative filtering methods, memory based and model based. **Memory-based models** are based on similarity between items or users, where we use cosine-similarity. **Model-based CF** is based on matrix factorization where we use SVD to factorize the matrix. SVD in recommender systems leverages matrix factorization to break down the user-item interaction matrix into latent factors. These factors are then used to predict missing ratings and make personalized recommendations by capturing the underlying relationships in user preferences and item characteristics. This approach is particularly effective in handling large and sparse datasets, making it a cornerstone of modern recommender systems.However, if little data is provided, this method becomes less effective.

To improve on this system, implementing a user interface which displays the recommended movies would be the next step. There are also many other methods to get the recommendations. By impementing them as well, I can see which method gives the most accurate recommendations